# Enunciado

1)   Se cuenta con un RDD con información sobre patentamiento
s 
de autos con la siguiente información `(patente, marca, modelo, 
versión, tipo_vehiculo, provincia, fecha)`, donde `tipo_vehiculo`
indica si la unidad patentada es auto, pickup, camión o moto. 
Se pide generar un programa en pySpark que **indique la marca y 
modelo del auto más patentado por tipo de vehículo en la pro
vincia 
de Buenos Aires en el mes de Abril de 2017**. (\*\*\*) (15 pts)

---

# Filtramos los datos

Nos quedamos con los datos de la provincia de Buenos Aires en el mes de Abril de 2017:

```
patentes_rdd = patentes_rdd.filter(patentes_rdd[5] == 'Buenos Aires')
                           .filter(patentes_rdd[6].contains('2017-04'))
```

Supusimos que el formato de `fecha` era de `YYYY-MM-DD`

# Pasamos a formato (K,V)

Como nos piden información por tipo de vehículo hacemos de `tipo_vehiculo` parte del value.
Por otra parte como queremos saber la cantidad de patentados por marca y modelo de los autos hacemos del key `('marca','modelo')`.
Por último hacemos parte del value un 1 por registro para sacar la cantidad por auto.

```
patentes_rdd = patentes_rdd.map(lambda x: ((x[1],x[2]),(x[4]),1)))
```

La estructura final nos queda de la forma: `(('marca','modelo'),('tipo_vehiculo',1))`

# Contamos la cantidad de patentados por auto (marca, modelo)

Tenemos que contar la cantidad por clave, por lo que podemos usar un `reduceByKey`.
Ahora bien, más adelante nos va a interesar trabajar con el tipo de vehiculo como key, así que aprovechamos y lo ponemos como key, pasando a (marca,modelo) a value. Lo mantenemos porque después nos pide devolver esta información.

```
patentes_rdd = patentes_rdd.reduceByKey(lambda x,y: (x[1][0],(x[0],x[1][1]+y[1][1])))
```

La estructura que nos queda es de la forma: `('tipo_vehiculo',(('marca´,'modelo'),cantidad))`

# Obtenemos (marca,modelo) del más patentado por tipo de vehículo

```
patente_rdd.reduceByKey(lambda x,y: (x[0],x[1][0]) if (x[1][1] > y[1][1]) else (x[0],y[1][0]))\
           .collect()
```

Lo que estamos devolviendo tiene el formato: `('tipo_vehiculo',('marca','modelo'))`